In [ ]:
from google.colab import drive
from google.colab import files
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
import json
import pandas as pd
import re

In [ ]:
os.listdir("gdrive/MyDrive/BT4222/Data")

['22FEB_fakenewsnet_dataset.zip', 'politifact.json', 'gossipcop.json']

In [ ]:
politifact_data = json.load(open("gdrive/MyDrive/BT4222/Data/politifact.json", "r"))
gossipcop_data = json.load(open("gdrive/MyDrive/BT4222/Data/gossipcop.json", "r"))

### Functions for processing

In [ ]:
!pip install contractions

     |████████████████████████████████| 284 kB 4.9 MB/s 
     |████████████████████████████████| 106 kB 63.2 MB/s 


In [ ]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9 MB 1.2 MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=b17c38b282873da8a4a3f444603ba03a6d342995418812ec979d25d6e8f780e8
  Stored in directory: /tmp/pip-ephem-wheel-cache-rg853sn8/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
import contractions
import spacy
import string
from spacy.tokenizer import Tokenizer
from spacy.util import compile_infix_regex

In [ ]:
PUNCT = string.punctuation + "’"

In [ ]:
def remove_tags(text):
    text = re.sub("\n",' ', text)
    text = re.sub("\'", "'", text)
    text = re.sub(' +', ' ', text)
    return text

In [ ]:
def expand_contractions(text):
  # creating an empty list
  expanded_words = []   
  for word in text.split():
    # using contractions.fix to expand the shortened words
    expanded_words.append(contractions.fix(word))  
    
  expanded_text = ' '.join(expanded_words)
  return expanded_text

In [ ]:
def pipeline():
    nlp = spacy.load("en_core_web_lg")
    inf = list(nlp.Defaults.infixes)               # Default infixes
    inf.remove(r"(?<=[0-9])[+\-\*^](?=[0-9-])")    # Remove the generic op between numbers or between a number and a -
    inf = tuple(inf)                               # Convert inf to tuple
    infixes = inf + tuple([r"(?<=[0-9])[+*^](?=[0-9-])", r"(?<=[0-9])-(?=-)"])  # Add the removed rule after subtracting (?<=[0-9])-(?=[0-9]) pattern
    infixes = [x for x in infixes if '-|–|—|--|---|——|~' not in x] # Remove - between letters rule
    infix_re = compile_infix_regex(infixes)

    custom_tokenizer = Tokenizer(nlp.vocab, prefix_search=nlp.tokenizer.prefix_search,
                                suffix_search=nlp.tokenizer.suffix_search,
                                infix_finditer=infix_re.finditer,
                                token_match=nlp.tokenizer.token_match,
                                rules=nlp.Defaults.tokenizer_exceptions)

    nlp.tokenizer = custom_tokenizer
    return nlp

def lemma(text, nlp):
    lemma = ' '.join([token.lemma_ if token.lemma_ != '-PRON-' else token.text for token in nlp(text)])
    return lemma

In [ ]:
def stopword_removal(text, nlp, stop=None):
    all_stopwords = nlp.Defaults.stop_words
    if stop:
        for word in stop:
            all_stopwords.add(word)
    doc = nlp(text)
    tokens_without_sw= [token.text for token in doc if not token.text in all_stopwords]
    filtered_sentence = " ".join(tokens_without_sw)
    return filtered_sentence

In [ ]:
def remove_punctuation(text, nlp):
    doc = nlp(text)
    return ' '.join([token.text for token in doc if token.text not in PUNCT])

In [ ]:
def clean_pipe(text, nlp):
    text = text.lower()
    text = remove_tags(text)
    text = expand_contractions(text)
    text = lemma(text, nlp)
    text = stopword_removal(text, nlp)
    text = remove_punctuation(text, nlp)
    return text

In [ ]:
nlp = pipeline()

### Clean politifact

In [ ]:
for article in politifact_data:
    article['title_clean'] = clean_pipe(article['title'], nlp)
    article['text_clean'] = clean_pipe(article['text'], nlp)

with open("gdrive/MyDrive/BT4222/Data/politifact_clean.json", 'w') as f:
  json.dump(politifact_data,f)

### Clean Gossipcop

In [ ]:
for article in gossipcop_data:
    article['title_clean'] = clean_pipe(article['title'], nlp)
    article['text_clean'] = clean_pipe(article['text'], nlp)

with open("gdrive/MyDrive/BT4222/Data/gossipcop_clean.json", 'w') as f:
  json.dump(gossipcop_data,f)